In [1]:
#импортируем библиотеки
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from math import floor
import os

In [3]:
#загрузим датасет
transactions = pd.read_csv('transactions.csv')
#отсортируем датасет
transactions.sort_values('order_id',inplace=True)

In [4]:
#загрузим датасет
product = pd.read_csv('products.csv')

In [5]:
#обьеденим датасеты
full_df = transactions.merge(product,how='inner',left_on='product_id',right_on='product_id')

In [6]:
#создадим копию
temp_df = full_df.copy()

In [7]:
#оставим только нужные столбцы
temp_df = temp_df[['order_id','user_id']]

In [9]:
#получим уникальных пользователей
unique_user_id = temp_df.user_id.unique()

In [12]:

def train_test_split(df,unique_user_id,percent_train=0.7):
    """
    Функция для разбиение чеков по пользователю в процентном соотношении и попорядку
    для ускорения работы функции каждые 5 тысяч пользователей сохраняются в отдельный файл
    """
  back = 0
  for list_unique_users in range(5000,100001,5000):
    df_train = pd.DataFrame(columns=df.columns)
    df_test = pd.DataFrame(columns=df.columns)
    if back == 0:
      unique_list = unique_user_id[:list_unique_users]
      back = list_unique_users
      for user in tqdm(unique_list):
        usr_one = df[df['user_id'] == user]
        list_transactions = usr_one.order_id.unique()
        transactions_train = list_transactions[:floor(len(list_transactions) * percent_train)]
        transactions_test = list_transactions[floor(len(list_transactions) * percent_train):]
        for order_id in transactions_train:
          df_train = df_train.append({'user_id':user,'order_id':order_id},ignore_index=True)
        for order_id in transactions_test:
          df_test = df_test.append({'user_id':user,'order_id':order_id},ignore_index=True) 
      df_train.to_csv('df_train{}.csv'.format(back))   
      df_test.to_csv('df_test{}.csv'.format(back))   
    else:
      unique_list = unique_user_id[back:list_unique_users]
      back += 5000
      for user in tqdm(unique_list):
        usr_one = df[df['user_id'] == user]
        list_transactions = usr_one.order_id.unique()
        transactions_train = list_transactions[:floor(len(list_transactions) * percent_train)]
        transactions_test = list_transactions[floor(len(list_transactions) * percent_train):]
        for order_id in transactions_train:
          df_train = df_train.append({'user_id':user,'order_id':order_id},ignore_index=True)
        for order_id in transactions_test:
          df_test = df_test.append({'user_id':user,'order_id':order_id},ignore_index=True) 
      df_train.to_csv('df_train{}.csv'.format(back))   
      df_test.to_csv('df_test{}.csv'.format(back))  

In [40]:
#создадим тренировачный и тестовый датасет
train = pd.DataFrame()
test = pd.DataFrame()

In [1]:
#вызовем функцию
train_test_split(full_df,unique_user_id)

In [2]:
# !unzip '/content/drive/MyDrive/train.zip'
# !unzip '/content/drive/MyDrive/test.zip'

In [18]:
#укажем пути для тренировачного и тестового датасета
train_patch = './train/'
test_patch = './test/'

In [19]:
#получим список файлов
train_files = os.listdir(train_patch)
test_files = os.listdir(test_patch)

In [41]:
#чиатем в цикле датасеты и обьеденяем
for i in tqdm(train_files):
  temp = pd.read_csv(train_patch + i,index_col='Unnamed: 0')
  train = train.append(temp)

  0%|          | 0/20 [00:00<?, ?it/s]

In [48]:
#меняем типы столбцов и оставляем только нужные колонки
train = train.astype({'order_id':'int','user_id':'int'})
train = train[['order_id','user_id']]

In [43]:
#чиатем в цикле датасеты и обьеденяем
for i in tqdm(test_files):
  temp = pd.read_csv(test_patch + i,index_col='Unnamed: 0')
  test = test.append(temp)

  0%|          | 0/20 [00:00<?, ?it/s]

In [3]:
#меняем типы столбцов и оставляем только нужные колонки
test = test.astype({'order_id':'int','user_id':'int'})
test = test[['order_id','user_id']]

In [51]:
#объединим с основным
train = train.merge(transactions,how='inner')

In [53]:
#объединим с основным
test = test.merge(transactions,how='inner')

In [55]:
#выгрузим тренировачный датасет
train.to_csv('train.csv')

In [56]:
#выгрузим тестовый датасет
test.to_csv('test.csv')